
<h2 style="text-align: center; font-family: Verdana; font-size: 24px; font-style: normal; font-weight: bold; text-decoration: underline; text-transform: none; letter-spacing: 2px; color: navy; background-color: #ffffff;"> Underwater img enhancement</h2>


<p align="center">
  
</p>


# 📌**Introduction:**
<!-- <h1 style="font-family: times-new-roman">📌Introduction</h1> -->
> <p style="font-family: times-new-roman">Robust recovery of lost colors in underwater images remains a challenging problem. We recently showed that this was partly due to the prevalent use of an atmospheric image formation model for underwater images. We proposed a physically accurate model that explicitly showed: 1) the attenuation coefficient of the signal is not uniform across the scene but depends on object range and reflectance, 2) the coefficient governing the increase in backscatter with distance differs from the signal attenuation coefficient. Here, we present a method that recovers color with the revised model using RGBD images. The Sea-thru method first calculates backscatter using the darkest pixels in the image and their known range information. Then, it uses an estimate of the spatially varying illuminant to obtain the range-dependent attenuation coefficient. Using more than 1,100 images from two optically different water bodies, which we make available, we show that our method outperforms those using the atmospheric model. Consistent removal of water will open up large underwater datasets to powerful computer vision and machine learning algorithms, creating exciting opportunities for the future of underwater exploration and conservation</p>

# 📑 **About the Notebook:**
> <p style="font-family: times-new-roman">there were total 8 methods for image color Enhancement.</p>

> - [8 Methods on Underwater Image Enhancement and Color Restoration]

> ## ⭕ **Underwater Image Enhancement**
> > - **CLAHE**: Contrast limited adaptive histogram equalization (1994)
> > - **Fusion-Matlab**: Enhancing underwater images and videos by fusion (2012)
> > - **GC**: Gamma Correction
> > - **HE**: Image enhancement by histogram transformation (2011)
> > - **ICM**: Underwater Image Enhancement Using an Integrated Colour Model (2007)
> > - **UCM**: Enhancing the low-quality images using Unsupervised Colour Correction Method (2010)
> > - **RayleighDistribution**: Underwater image quality enhancement through composition of dual-intensity images and Rayleigh-stretching (2014)
> > - **RGHS**: Shallow-Water Image Enhancement Using Relative Global Histogram Stretching Based on Adaptive Parameter Acquisition (2018)






> > - Dataset: http://csms.haifa.ac.il/profiles/tTreibitz/datasets/sea_thru/index.html

> > <p align="center">
<img width = "600" src="https://i.postimg.cc/6qpZCTFK/sea-thru3.jpg">
</p>



> ## 🧪 **Importance of image Pre-processing:**
> > <p style="font-family: times-new-roman"> Image preprocessing in one of the most important part of a Model building pipeline. It helps to improve the quality of your image, we apply different kind of filters and methods to enhance sertain aspect of the image, may be the given image is very blurry, or may be the image in very noisy, or maybe the image size across the data is not similar, or may be the labeled data is not accurate etc. In these cases we need to use Image preprocessing to clean the data. In some cases it also helps by decreasing model training time and increasing model inference speed.  </p>

</b>


> > > <p align="center">
<img src="https://i.imgur.com/wc9rY2J.png">
</p>

> > > <p align="center">
<img src="https://i.imgur.com/ho1D0Ye.png">
</p>

> > > <p align="center">
<img src="https://i.imgur.com/pwBNYJr.png">
</p>



# 📚 **Importing Libraries:**

In [ ]:
!pip install -q ffmpeg-python

In [ ]:
import warnings
warnings.filterwarnings('ignore')
import cv2
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import matplotlib.pyplot as plt
import seaborn as sns
import random
import ffmpeg
from IPython.display import Video
from tqdm import tqdm
import tensorflow as tf
import tensorflow_addons as tfa
import logging
from itertools import cycle

logging.disable(logging.WARNING)
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 


plt.style.use('ggplot')
cm = sns.light_palette("green", as_cmap=True)
pd.option_context('display.max_colwidth', 100)
color_pal = plt.rcParams["axes.prop_cycle"].by_key()["color"]
color_cycle = cycle(plt.rcParams["axes.prop_cycle"].by_key()["color"])

In [ ]:
# SEED EVERYTHING
random.seed(hash("setting random seeds") % 2**32 - 1)
np.random.seed(hash("improves reproducibility") % 2**32 - 1)

# config
class config:
    BASE_DIR = "../input/tensorflow-great-barrier-reef/train_images/"

# 🔎 **Lets checkout the training data:**

In [ ]:
img_og = plt.imread('../input/tensorflow-great-barrier-reef/train_images/video_1/9101.jpg')
img_9101 = cv2.imread('../input/tensorflow-great-barrier-reef/train_images/video_1/9101.jpg')

In [ ]:
df = pd.read_csv('../input/tensorflow-great-barrier-reef/train.csv')
train_dir = "../input/tensorflow-great-barrier-reef/train_images"
df['image_path'] = train_dir + "/video_" + df['video_id'].astype(str) + "/" + df['video_frame'].astype(str) + ".jpg"
df.head().style.set_properties(**{'background-color': 'black',
                           'color': 'lawngreen',
                           'border-color': 'white'})

In [ ]:
df.info() # lets check more details about the data

In [ ]:
df[df.annotations.str.len() > 2].head(5).style.background_gradient(cmap=cm) # filling up the annotation column

In [ ]:
df['annotations'] = df['annotations'].apply(eval)
df_train_v2 = df[df.annotations.str.len() > 0 ].reset_index(drop=True)
df_train_v2.head(5).style.background_gradient(cmap='Reds')

# What is Sequence and its properties:

In [ ]:
df_train_v2["no_of_bbox"] = df_train_v2["annotations"].apply(lambda x: len(x))
df_train_v2["sequence"].value_counts(), len(df_train_v2["sequence"].value_counts())

In [ ]:
for i in range(3):
    print(df_train_v2["sequence"][df_train_v2["video_id"] == i].unique(), 
          df_train_v2["sequence"][df_train_v2["video_id"] == i].nunique())

# Bounding box analysis in each video:

In [ ]:
def plot_with_count(df,vid):
    names = df["bbox_typ"].to_list()
    values = df["counts"].to_list()

    N = len(names)
    menMeans = values
    ind = np.arange(N)

    plt.rcParams["figure.figsize"] = [7.00, 3.50]
    plt.rcParams["figure.autolayout"] = True
    fig, ax = plt.subplots(figsize=(15,6))

    ax.bar(ind,menMeans,width=0.4)
    plt.xticks(np.arange(0, N, step=1))
    plt.title(f"Number of bounding box VS Count of Bounding Box: Video{vid} ",fontsize=20)

    plt.xlabel('Number of bounding box', fontsize=18)
    plt.ylabel('Count', fontsize=16)

    for index,data in enumerate(menMeans):
        plt.text(x=index , y =data+1 , s=f"{data}" , fontdict=dict(fontsize=15))

In [ ]:
vid = 0
df_vod0_bbox_cnt = df_train_v2["no_of_bbox"][df_train_v2["video_id"] == vid].value_counts().reset_index() # LEARNING .to_frame() and .reset_index()
df_vod0_bbox_cnt.columns = ['bbox_typ', 'counts']
plot_with_count(df_vod0_bbox_cnt,vid)

In [ ]:
vid = 1
df_vod1_bbox_cnt = df_train_v2["no_of_bbox"][df_train_v2["video_id"] == vid].value_counts().reset_index() # LEARNING .to_frame() and .reset_index()
df_vod1_bbox_cnt.columns = ['bbox_typ', 'counts']
plot_with_count(df_vod1_bbox_cnt,vid)

In [ ]:
vid = 2
df_vod2_bbox_cnt = df_train_v2["no_of_bbox"][df_train_v2["video_id"] == vid].value_counts().reset_index() # LEARNING .to_frame() and .reset_index()
df_vod2_bbox_cnt.columns = ['bbox_typ', 'counts']
plot_with_count(df_vod2_bbox_cnt,vid)

In [ ]:
# https://www.kaggle.com/julian3833/reef-a-cv-strategy-subsequences
df = pd.read_csv("/kaggle/input/tensorflow-great-barrier-reef/train.csv")
df['annotations'] = df['annotations'].apply(eval)
df['n_annotations'] = df['annotations'].str.len()
df['has_annotations'] = df['annotations'].str.len() > 0
df['has_2_or_more_annotations'] = df['annotations'].str.len() >= 2
df['doesnt_have_annotations'] = df['annotations'].str.len() == 0
df['image_path'] = config.BASE_DIR + "video_" + df['video_id'].astype(str) + "/" + df['video_frame'].astype(str) + ".jpg"

In [ ]:
df_agg = df.groupby(["video_id", 'sequence']).agg({'sequence_frame': 'count', 'has_annotations': 'sum', 'doesnt_have_annotations': 'sum'})\
           .rename(columns={'sequence_frame': 'Total Frames', 'has_annotations': 'Frames with at least 1 object', 'doesnt_have_annotations': "Frames with no object"})
df_agg

The point I want to make over here is that, In the pipeline we need to add more images with no lables, because there are many images in the private test case which has zero objects associated with it. But we cant actually infuse some unlabeled/no object image into the pipeline coz there are 80% of the total dataset present in that unlabeled/no object iamge categoury. So to do that we first need to choose a percentage in which we want to make the combiniation of both labeled and unlabeled image. So, say we deceide to take 6k img where 5k has object and 1k has not. Now the problem comes down to how to choose these 1k images. I would say choose few sequence and keep all of the unlabeled/no object images in the dataset and do that untill you get near 1k images. You might need to hand pich for each fold. I was thinking about using the fold CSVs generated from julian's NB and instade of filtering out all the unlabeled/no object images, keep some of them and train the whole network using that.

### Future work:
- Add plots on distribution of height and width of the bbox provided
- Add plots on distribution of area of the bbox provided

# 🎨 **Some Data Preparation:**

In [ ]:
def RecoverCLAHE(sceneRadiance):
    clahe = cv2.createCLAHE(clipLimit=7, tileGridSize=(14, 14))
    for i in range(3):
        sceneRadiance[:, :, i] = clahe.apply((sceneRadiance[:, :, i]))
    return sceneRadiance

dest_path1 = "./clahe_img"
os.mkdir(dest_path1)

for img_path in tqdm(df_train_v2["image_path"][0:400]):

    image = plt.imread(img_path)
    image_cv = cv2.imread(img_path)
    img_clahe = RecoverCLAHE(image_cv)
    file_name = img_path.split("/")[-1]
    
    cv2.imwrite(dest_path1+"/"+file_name, img_clahe)

In [ ]:
dest_path1 = "./annot_img"
os.mkdir(dest_path1)

idx = 0
for img_idx in tqdm(df_train_v2["image_path"][0:400]):
    file_name = img_idx.split("/")[-1] 
    img_path = os.path.join("./clahe_img",file_name)
    image = plt.imread(img_path)


    for i in range(len(df_train_v2["annotations"][idx])):
        file_name = img_path.split('/')[-1]
        b_boxs = df_train_v2["annotations"][idx][i]
        x,y,w,h = b_boxs["x"],b_boxs["y"],b_boxs["width"],b_boxs["height"]

        image = cv2.rectangle(image, (x, y), (x + w, y + h), (36,255,12), 3)
        image = cv2.putText(image, 'starfish', (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (36,255,12), 2)

    cv2.imwrite(dest_path1+"/"+file_name, image)
    idx +=1

# 🧽 ** about the images:**

In [ ]:
plt.figure(figsize = (12,15))


plt.rcParams["figure.figsize"] = [20.00, 10.50]
plt.rcParams["figure.autolayout"] = True

fig, ax = plt.subplots(figsize=(20,6))

ax.imshow(plt.imread("../input/tensorflow-great-barrier-reef/train_images/video_0/1068.jpg"));
newax = fig.add_axes([0.3,0.3,0.6,0.7], anchor='NE', zorder=1)


newax.axis('off')
plt.show();

### Lets check we have images with same size or not:

In [ ]:
img_sizes = []
for i in df_train_v2["image_path"]:
    img_sizes.append(plt.imread(i).shape)

np.unique(img_sizes)

In [ ]:
# lets check total number of images with annotations
len(df_train_v2)

# 🦀 ** about the annotations:** 

In [ ]:
plt.figure(figsize = (12,15))


plt.figure(figsize = (12,15))


plt.rcParams["figure.figsize"] = [20.00, 10.50]
plt.rcParams["figure.autolayout"] = True

fig, ax = plt.subplots(figsize=(20,6))

ax.imshow(plt.imread("./annot_img/40.jpg"))

newax = fig.add_axes([0.26,0.2,0.6,0.6], anchor='NE', zorder=1)


newax.axis('off')
plt.show()

# 📪 **80% of the data has not objects:** 

In [ ]:
count_bbox = []
for i in df["annotations"]:
    count_bbox.append(len(i))
    
from collections import defaultdict


bbox_dict = defaultdict(int)

for val in count_bbox:
    bbox_dict[val] += 1
    

names = list(bbox_dict.keys())
values = list(bbox_dict.values())

N = len(list(bbox_dict.values()))
menMeans = list(bbox_dict.values())
ind = np.arange(N)

plt.rcParams["figure.figsize"] = [20.00, 10.50]
plt.rcParams["figure.autolayout"] = True
im = plt.imread('../input/random-images-dataset/Number of bounding box VS Count of Bounding Boxlittle one.png') # insert local path of the image.
fig, ax = plt.subplots(figsize=(20,6))

ax.bar(ind,menMeans,width=0.4)
plt.xticks(np.arange(0, N, step=1))
plt.title("Number of bounding box VS Count of Bounding Box",fontsize=20)

plt.xlabel('Number of bounding box', fontsize=18)
plt.ylabel('Count', fontsize=16)
for index,data in enumerate(menMeans):
    plt.text(x=index , y =data+1 , s=f"{data}" , fontdict=dict(fontsize=20))
newax = fig.add_axes([0.3,0.35,0.6,0.5], anchor='NE', zorder=1)
newax.imshow(im)

newax.axis('off')

plt.show()

> <font size="4" face="verdana">
            <b>"professor" Squidward:</b>  Well seem like there are more empty bounding boxes, the fill ones.
        </font>
        
<center><div class="alert alert-block alert-warning" style="margin: 2em; line-height: 1.7em; font-family: Verdana;">
    There are alot of things to discover from this dataset. So keep a eye on the EDA part Im surely going to update that.
</div></center>

# 📊 **How to perform histogram equalization?**
Histogram Equalization is a method of contrast adjustment based on the image's histogram.

> In this image the pixel values are are between 0-255, but we will not find any pixel values which are exactly 0 or 255, it does not have any image which is pure white or pure black.If we apply the histogram equalization then it will reduce the color depth.Currently the minimum pixel value is 52 and the highest is 255.
After you apply histogram equalization, you will find the the min pixel value now got transformed to zero and the max got converted to 255. So, notice again how the min and max values are equalized between 0 and 255, we also see less shade of gray.(view fig1 to 2)

<img src="https://i.imgur.com/mWTXqZ4.png" style="width:500px;height:250px;">
<center><p style="padding-left:380px;color:red">Fig1:before applying histogram equalization</p></center>

<img src="https://i.imgur.com/5diUO7c.png" style="width:500px;height:250px;">
<center><p style="padding-left:380px;color:red">Fig2: After applying histogram equalization</p></center>

> Again, now we have a image on the left hand side and it's coresponding histogram(in red) on the right the black line is nothing but the cumulative of the pixel values.So, after we apply the Histogram equalizer that cumulative changes to linear step function. Notice that we don't literally flatten out the histogram we only just focus on the cumulative linear. And the real mathematics behind the Histogram equalization is just like that.(view fig 3 to 5)
<img src="https://i.imgur.com/uIBnzbu.png" style="width:500px;height:250px;">

<center><p style="padding-left:380px;color:red">Fig3:before applying histogram equalization</p></center>

<img src="https://i.imgur.com/fuIGrCi.png" style="width:500px;height:250px;">

<center><p style="padding-left:380px;color:red">Fig4: After applying histogram equalization</p></center>

<img src="https://i.imgur.com/9q3NVIg.png" style="width:500px;height:250px;">
<center><p style="padding-left:380px;color:red">Fig5:Main difference </p></center>

We mainly use histogram equalization when we need to increase the contrast of the image.

In [ ]:
def he_hsv(img_demo):
    img_hsv = cv2.cvtColor(img_demo, cv2.COLOR_RGB2HSV)

    # Histogram equalisation on the V-channel
    img_hsv[:, :, 2] = cv2.equalizeHist(img_hsv[:, :, 2])

    # convert image back from HSV to RGB
    image_hsv = cv2.cvtColor(img_hsv, cv2.COLOR_HSV2RGB)
    
    return image_hsv

In [ ]:
def plot_img(img_dir,num_items,func,mode):
    img_list = random.sample(os.listdir(img_dir), num_items)

    for i in range(len(img_list)):
        full_path = img_dir + '/' + img_list[i]
        img_temp1 = plt.imread(full_path)
        img_temp_cv = cv2.imread(full_path)
        plt.figure(figsize=(20,15))
        plt.subplot(1,2,1)
        plt.imshow(img_temp1);
        plt.subplot(1,2,2)
        if mode == 'plt':
            plt.imshow(func(img_temp1));
        elif mode == 'cv2':
            plt.imshow(func(img_temp_cv));

In [ ]:
vid_0_dir = "../input/tensorflow-great-barrier-reef/train_images/video_0"
num_items1 = 4
plot_img(vid_0_dir,num_items1,he_hsv,"plt")

# 🧩 **HE code from the repo:**

- below is the code from the repo
- check out the Code from the repo [here](https://github.com/wangyanckxx/Single-Underwater-Image-Enhancement-and-Color-Restoration/tree/master/Underwater%20Image%20Enhancement/HE )
- It is basically allpying Histogram Equalizers in each channel

In [ ]:
def RecoverHE(sceneRadiance):
    for i in range(3):
        sceneRadiance[:, :, i] =  cv2.equalizeHist(sceneRadiance[:, :, i])
    return sceneRadiance

vid_0_dir = "../input/tensorflow-great-barrier-reef/train_images/video_0"
num_items1 = 4
plot_img(vid_0_dir,num_items1,RecoverHE,"cv2")

# 🔮 **CLAHE: without repo code:**

> Contrast Limited AHE (CLAHE) differs from adaptive histogram equalization in its contrast limiting. In the case of CLAHE, the contrast limiting procedure is applied to each neighborhood from which a transformation function is derived. CLAHE was developed to prevent the over amplification of noise that adaptive histogram equalization can give rise to.

> We apply CLAHE to color images, where usually it is applied on the luminance channel and the results after equalizing only the luminance channel of an HSV image are much better than equalizing all the channels of the BGR image. 

<div class="alert alert-block alert-info">
<b>Note:</b> Each pixel in a image has brightness level, called luminance. This value is between 0 to 1, where 0 means complete darkness (black), and 1 is brightest (white)
</div>



In [ ]:
def clahe_hsv(img):
    hsv_img = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)

    h, s, v = hsv_img[:,:,0], hsv_img[:,:,1], hsv_img[:,:,2]
    clahe = cv2.createCLAHE(clipLimit = 15.0, tileGridSize = (20,20))
    v = clahe.apply(v)

    hsv_img = np.dstack((h,s,v))

    rgb = cv2.cvtColor(hsv_img, cv2.COLOR_HSV2RGB)
    
    return rgb


vid_0_dir = "../input/tensorflow-great-barrier-reef/train_images/video_0"
num_items1 = 4
plot_img(vid_0_dir,num_items1,clahe_hsv,"cv2")

# 🥏 **CLAHE: with repo code:**

In [ ]:
def RecoverCLAHE(sceneRadiance):
    clahe = cv2.createCLAHE(clipLimit=7, tileGridSize=(14, 14))
    for i in range(3):

        
        sceneRadiance[:, :, i] = clahe.apply((sceneRadiance[:, :, i]))


    return sceneRadiance

vid_0_dir = "../input/tensorflow-great-barrier-reef/train_images/video_0"
num_items1 = 4
plot_img(vid_0_dir,num_items1,RecoverCLAHE,"cv2")

> <font size="4" face="verdana">
            From the image, it seems like all the rocks, plants, and other underwater objects are on the ground and sunlight falling upon them, not inside a sea.
        </font>

# 🪀 **GC: without repo code:**

> Different camera or video recorder devices do not correctly capture luminance. (they are not linear) Different display devices (monitor, phone screen, TV) do not display luminance correctly neither. So, one needs to correct them, therefore the gamma correction function. Gamma correction function is a function that maps luminance levels to compensate the non-linear luminance effect of display devices (or sync it to human perceptive bias on brightness). For example check out the below image,


<p align="center">
    <img width="600" src="https://i.imgur.com/2GLA50Q.png">
</p>


<div class="alert alert-block alert-info">
<b>Source:</b> https://medium.com/giscle/how-we-utilized-gamma-correction-for-increasing-our-training-data-47c16a040adc
<br>
<b>Check out more about GC, here :</b> https://en.wikipedia.org/wiki/Gamma_correction
</div>


In [ ]:
def gamma_enhancement(image,gamma):
    R = 255.0
    return (R * np.power(image.astype(np.uint32)/R, gamma)).astype(np.uint8)

plt.figure(figsize=(20,15))
plt.subplot(2,2,1)
plt.imshow(img_og);
plt.subplot(2,2,2)
plt.imshow(gamma_enhancement(img_9101,1/0.6))

plt.subplot(2,2,3)
plt.imshow(img_og);
plt.subplot(2,2,4)
plt.imshow(gamma_enhancement(img_og,1/0.6))


# 🖌 **GC: with repo code:**

In [ ]:
def RecoverGC(sceneRadiance):
    sceneRadiance = sceneRadiance/255.0
    
    for i in range(3):
        sceneRadiance[:, :, i] =  np.power(sceneRadiance[:, :, i] / float(np.max(sceneRadiance[:, :, i])), 3.2)
    sceneRadiance = np.clip(sceneRadiance*255, 0, 255)
    sceneRadiance = np.uint8(sceneRadiance)
    return sceneRadiance

vid_0_dir = "../input/tensorflow-great-barrier-reef/train_images/video_0"
num_items1 = 4
plot_img(vid_0_dir,num_items1,RecoverGC,"cv2")

# 📍 **ICM: with repo code**

In [ ]:
import numpy as np

def global_stretching(img_L,height, width):
    I_min = np.min(img_L)
    I_max = np.max(img_L)
    I_mean = np.mean(img_L)

    array_Global_histogram_stretching_L = np.zeros((height, width))
    for i in range(0, height):
        for j in range(0, width):
            p_out = (img_L[i][j] - I_min) * ((1) / (I_max - I_min))
            array_Global_histogram_stretching_L[i][j] = p_out

    return array_Global_histogram_stretching_L

def stretching(img):
    height = len(img)
    width = len(img[0])
    for k in range(0, 3):
        Max_channel  = np.max(img[:,:,k])
        Min_channel  = np.min(img[:,:,k])
        for i in range(height):
            for j in range(width):
                img[i,j,k] = (img[i,j,k] - Min_channel) * (255 - 0) / (Max_channel - Min_channel)+ 0
    return img

from skimage.color import rgb2hsv,hsv2rgb
import numpy as np



def  HSVStretching(sceneRadiance):
    height = len(sceneRadiance)
    width = len(sceneRadiance[0])
    img_hsv = rgb2hsv(sceneRadiance)
    h, s, v = cv2.split(img_hsv)
    img_s_stretching = global_stretching(s, height, width)

    img_v_stretching = global_stretching(v, height, width)

    labArray = np.zeros((height, width, 3), 'float64')
    labArray[:, :, 0] = h
    labArray[:, :, 1] = img_s_stretching
    labArray[:, :, 2] = img_v_stretching
    img_rgb = hsv2rgb(labArray) * 255

    

    return img_rgb

def sceneRadianceRGB(sceneRadiance):

    sceneRadiance = np.clip(sceneRadiance, 0, 255)
    sceneRadiance = np.uint8(sceneRadiance)

    return sceneRadiance

In [ ]:
def RecoverICM(img1):
    img = stretching(img1)
    sceneRadiance = sceneRadianceRGB(img)
    sceneRadiance = HSVStretching(sceneRadiance)
    sceneRadiance = sceneRadianceRGB(sceneRadiance)
    
    return sceneRadiance


vid_0_dir = "../input/tensorflow-great-barrier-reef/train_images/video_0"
num_items1 = 4
plot_img(vid_0_dir,num_items1,RecoverICM,"cv2")

# 🎯 **Main Working Code:**
> I am using **`tfa.image.equalize`** for the preprocessing. You can also add this like that in the data augmentation pipeline. Im passing this as a function in the **`plot_img_tf`** function.

In [ ]:
def plot_img_tf(img_dir,num_items,func,mode):
    img_list = random.sample(os.listdir(img_dir), num_items)
    full_path = img_dir + '/' + img_list[0]
    img_temp_plt = plt.imread(full_path)
    img_temp_cv = cv2.imread(full_path)
    if mode=="plt":
        
        img_stack = np.hstack((img_temp_plt,func(img_temp_plt)))
        plt.figure(figsize=(20,15))
        plt.imshow(img_stack);
        plt.title("Original Image VS Enhanced Image",fontsize=25)
        plt.axis("off")
        plt.show()
    if mode=="cv2":
        
        img_stack = np.hstack((img_temp_cv,func(img_temp_cv)))
        plt.figure(figsize=(20,15))
        plt.imshow(img_stack);
        plt.title("Original Image VS Enhanced Image",fontsize=25)
        plt.axis("off")
        plt.show()
    
    
    for i in range(1, len(img_list)):
        full_path = img_dir + '/' + img_list[i]
        img_temp_plt = plt.imread(full_path)
        img_temp_cv = cv2.imread(full_path)
        if mode=="plt":
            img_stack = np.hstack((img_temp_plt,func(img_temp_plt)));
            plt.figure(figsize=(20,15))
            plt.imshow(img_stack);
            plt.axis("off")
            plt.show()
        if mode=="cv2":
            img_stack = np.hstack((img_temp_cv,func(img_temp_cv)));
            plt.figure(figsize=(20,15))
            plt.imshow(img_stack);
            plt.axis("off")
            plt.show()

img_dir = "../input/tensorflow-great-barrier-reef/train_images/video_0"
num_items = 4
plot_img_tf(img_dir,num_items,tfa.image.equalize,"plt")

# 📺 **Lets checkout How it looks like as a video:**

In [ ]:
(
    ffmpeg.input('./clahe_img/*.jpg', pattern_type='glob', framerate=25)
    .output('img_movie.mp4')
    .run()
)

In [ ]:
(
    ffmpeg.input('./annot_img/*.jpg', pattern_type='glob', framerate=25)
    .output('annot_movie.mp4')
    .run()
)

In [ ]:
Video("./img_movie.mp4")

In [ ]:
Video("./annot_movie.mp4")